In [5]:
# Title: DMRG for Bachelor Thesis
# Auth'or: Aaron Sander
# Date: March 2020

# This program is used for initial learning of tensor network methods
# to be used in my bachelor thesis.
# It is an implementation of Matrix Product States (MPS) and Density Matrix
# Renormalization Group (DMRG) for finding the ground state of an arbitrary
# Hamiltonian

In [6]:
######################### IMPORTS #############################################
import numpy as np
import random as rand

In [7]:
######################## INITIALIZATIONS ##############################################
def initialize_random_state(numParticles, bondDim):
    # Dimensions (2 x d)
    M_1 = np.random.rand(2, bondDim)

    # Dimensions (d x d x 2)
    M_i = np.random.rand(bondDim, bondDim, 2)


    # Dimensions (2 x d)
    M_N = np.random.rand(2, bondDim)

    MPS = [M_1] + [M_i]*(numParticles-2) + [M_N]
    return MPS


def initialize_W_state(numParticles):
    # d = 2
    # Dimensions (2 x d)
    M_1 = np.array([np.array([1, 0]),
                    np.array([0, 1])])


    # Dimensions (d x d x 2)
    # (2 x d x d) -> (d x d x 2)
    M_i = np.array([np.array([[1, 0],
                              [0, 1]]),
                    np.array([[0, 1],
                              [0, 0]])])
    M_i = np.transpose(M_i, (1, 2, 0))

    # Dimensions (2 x d)
    M_N = np.array([np.array([[0],
                              [1]]),
                    np.array([[1],
                              [0]])])
    M_N = np.squeeze(M_N)

    MPS = [M_1] + [M_i]*(numParticles-2) + [M_N]
    return MPS


def initialize_GHZ_state(numParticles):
    # d = 2
    # Dimensions (2 x d)
    M_1 = np.array([[1, 0], [0, 1]])

    # Dimensions (d x d x 2)
    # (2 x d x d) -> (d x d x 2)
    M_i = np.array([np.array([[1, 0],
                              [0, 0]]),
                    np.array([[0, 0],
                              [0, 1]])])
    M_i = np.transpose(M_i, (1, 2, 0))

    # Dimensions (2 x d)
    M_N = np.array([np.array([[1],
                              [0]]),
                    np.array([[0],
                              [1]])])
    M_N = np.squeeze(M_N)
    
    MPS = [M_1] + [M_i]*(numParticles-2) + [M_N]
    return MPS


### Quantum Ising Model ###
def initialize_quantum_ising(numParticles):
    # Operators
    pauli_z = np.array([[1, 0],
                        [0, -1]])

    pauli_x = np.array([[0, 1],
                        [1, 0]])

    zero = np.zeros((2, 2))
    identity = np.identity(2)

    # Interaction parameter
    g = 2

    # Initialization of Hamiltonian MPO (entries done by hand)
    # Dimensions (1x3x2x2)->(3x2x2)
    left_bound = np.array([identity, pauli_z, g*pauli_x])

    # Dimensions (3x3x2x2)
    inner = np.array([np.array([identity, pauli_z, g*pauli_x]),
                    np.array([zero, zero, pauli_z]),
                    np.array([zero, zero, np.identity(2)])])

    # Dimensions (3x1x2x2)->3x2x2
    right_bound = np.array([[g*pauli_x],
                            [pauli_z],
                            [identity]])
    right_bound = np.squeeze(right_bound)  # Removes unnecessary index

    MPO = [left_bound] + [inner]*(N-2) + [right_bound]

    return MPO

In [1]:
###################### FUNCTIONS FOR MPO ##############################################
# Used to contract tensors horizontally from A->B
# All 2 dimensional tensors have dimensions (ij) or (ab)
# All 3 dimensional tensors have dimensions (ijk) or (abc)
# All 4 dimensional tensors have dimensions (ijkl) or (abcd)

# Examples:
# 4-tensor: Inner MPO
# 3-tensor: Inner MPS, Outer MPO, Half contracted inner MPO-MPS
# 2-tensor: Outer MPS, Half contracted outer MPO-MPS, Fully contracted lattice point

# This all works assuming the MPS has form (2 x d), (d x d x 2), (2 x d)
# Only valid for full system with MPS, MPO, MPS.
# How to read: If we are at tensor A, what other tensor B can I contract with?
def contract_horizontal(A, B, dir):
    if A.ndim == 3:
        if B.ndim == 4: 
            if dir =='right':
                tensor = np.einsum('ijk, ibcd->bjckd', A, B)
                # Reshape to (b, j*c, k*d)
                tensor = np.reshape(tensor, (B.shape[1], A.shape[1]*B.shape[2], A.shape[2]*B.shape[3]))
            elif dir == 'left':
                tensor = np.einsum('ijk, aicd->ajckd', A, B)
                # Reshape to (a, j*c, k*d)
                tensor = np.reshape(tensor, (B.shape[0], A.shape[1]*B.shape[2], A.shape[2]*B.shape[3]))
        elif B.ndim == 3: # Used for contraction of MPO itself
            if dir == 'right' or 'left':  # Can be removed, left for readability
                tensor = np.einsum('ijk, ibc->jbkc', A, B)
                # Reshape collapses indices to (j*b, k*c)
                tensor = np.reshape(tensor, (A.shape[1]*B.shape[1], A.shape[2]*B.shape[2]))
    
    elif A.ndim == 2:
        if B.ndim == 3:
            if dir == 'right':
                tensor = np.einsum('ij, jbc->icb', A, B)
                # Reshape to (i*c, b)
                tensor = np.reshape(tensor, (A.shape[0]*B.shape[2], B.shape[1]))
            elif dir == 'left':
                tensor = np.einsum('ij, ajc->ica', A, B)
                # Reshape to (i*c, a)
                tensor = np.reshape(tensor, (A.shape[0]*B.shape[2], B.shape[0]))
        elif B.ndim == 2:
            if dir == 'right' or 'left': # Direction independent since both MPS edges are (2 x d)
                tensor = np.einsum('ij, aj->ia', A, B)
                # Reshape to (i*b)
                tensor = np.reshape(tensor, (A.shape[0]*B.shape[0]))

    elif A.ndim == 1:
        if B.ndim == 2: # Final contraction before scalar product
            if dir == 'right':
                tensor = np.einsum('i, ib->b', A, B)
            elif dir == 'left':
                tensor = np.einsum('i, ai->a', A, B)
        elif B.ndim == 1: # Inner product
            tensor = np.einsum('i, i', A, B)

    return tensor


def contract_vertical(A, B, dir):
    if A.ndim == 3:
        if B.ndim == 4:
            if dir == 'down' or 'up':
                tensor = np.einsum('ijk, abkd->iajbd', A, B) # Contracts (d x d x 2) and (3 x 3 x 2 x 2)
                # Reshape to (i*a, j*b, d)
                tensor = np.reshape(tensor, (A.shape[0]*B.shape[0], A.shape[1]*B.shape[1], B.shape[3]))
        elif B.ndim == 3:
            if dir == 'down' or 'up': # Contract (3d x 3d x 2) and (d x d x 2)
                tensor = np.einsum('ijk, abk->iajb', A, B)
                # Reshape to (i*a, j*b)
                tensor = np.reshape(tensor, (A.shape[0]*B.shape[0], A.shape[1]*B.shape[1]))

    elif A.ndim == 2:
        if B.ndim == 3:
            if dir == 'down':  # From Bra->Operator->Ket
                tensor = np.einsum('ij, abi->jab', A, B) # Contract (2 x d) and (3 x 2 x 2)
                # Reshape to (j*a, b)
                tensor = np.reshape(tensor, (A.shape[1]*B.shape[0], B.shape[1]))
            elif dir == 'up':  # From Ket->Operator->Bra
                tensor = np.einsum('ij, aic->jac', A, B)
                # Reshape to (j*a, c)
                tensor = np.reshape(tensor, (A.shape[1]*B.shape[0], B.shape[2]))
        elif B.ndim == 2:
            if dir == 'down' or 'up':
                tensor = np.einsum('ij, jb->ib', A, B) # Contract (3d x 2) and (2 x d)
                # Reshape to (i*b)
                tensor = np.reshape(tensor, (A.shape[0]*B.shape[1]))

    return tensor

### Calculates the expectation value by vertical contraction then horizontal contraction ###
def calculate_expectation(MPS_bra, MPO, MPS_ket, vert_dir, horiz_dir):
    # Initialize list of tensors
    tensor = [None]*N

    # Contract <MPS|MPO|MPS> at each lattice position
    # Down: Bra -> MPO -> Ket
    # Up: Ket -> MPO -> Bra
    for i in range(0, N):
        if vert_dir == 'down':
            first_contraction = contract_vertical(MPS_bra[i], MPO[i], vert_dir)
            tensor[i] = contract_vertical(first_contraction, MPS_ket[i], vert_dir)
        if vert_dir == 'up':
            first_contraction = contract_vertical(MPS_ket[i], MPO[i], vert_dir)
            tensor[i] = contract_vertical(first_contraction, MPS_bra[i], vert_dir)

    # Contract each tensor created from above
    # Left and right necessary for scanning in DMRG
    if horiz_dir == 'right':
        E = tensor[0]
        for i in range(1, len(tensor)):
            E = contract_horizontal(E, tensor[i], horiz_dir)
    if horiz_dir == 'left':
        E = tensor[-1]
        for i in range(len(tensor)-2, -1, -1):
            E = contract_horizontal(E, tensor[i], horiz_dir)

    return E

# TODO: Determine if vertical/horizontal can be generalized and combined
# TODO: Cut singular values under a threshold
# NOTE: All directions of contractions have been tested and work correctly.
#       This was verified by checking if result is equal for going
#       left<->right and up<->down.

In [2]:
### CANONICAL FORMS ###

### Normalizes the tensor network such that A_dagger A = Identity ###
def left_normalize(MPS):
    lambda_tensors = []
    # Loop from left bound to right bound
    for i in range(0, len(MPS)):
        # Left bound (2 x d)
        if i == 0:
            M = MPS[i]
            # Dimensions (d x d), (d x 2), (2 x 2)
            U, S_vector, V = np.linalg.svd(M, full_matrices=False)
            S = np.diag(S_vector)

            MPS[i] = U
            # Contract such that M' = SVM
            MPS[i+1] = np.einsum('ij, jbc->ibc', S @ V, MPS[i+1])
        # Inner tensors (d x d x 2)
        elif i != len(MPS)-1:
            # Reshape such that (d x d x 2) -> (d x 2 x d) for correct reshape
            # Legs closest to each other collape
            M = np.transpose(MPS[i], (0, 2, 1))
            # Collapse left bond and physical dimensions
            M = np.reshape(M, (MPS[i].shape[0]*MPS[i].shape[2], MPS[i].shape[1]))
            U, S_vector, V = np.linalg.svd(M, full_matrices=False)
            S = np.diag(S_vector)

            # Dimensions according to d x 2 x d still
            # (Left bond, physical dimension, right bond)
            MPS[i] = np.reshape(U, (MPS[i].shape[0], MPS[i].shape[2], U.shape[1]))
            MPS[i] = np.transpose(MPS[i], (0, 2, 1)) # Reshape back to (d x d x 2)

            # Last tensor is a rank-2 tensor
            if i == len(MPS)-2:
                # Transpose due to convention of using 2 x d for last position
                MPS[i+1] = np.einsum('ij, jb->ib', S @ V, MPS[i+1].T).T 
            # Other inner tensors are rank-3 tensors
            else:
                MPS[i+1] = np.einsum('ij, jbc->ibc', S @ V, MPS[i+1])

        # Right bound (2 x d)
        elif i == len(MPS)-1:
            # No right bond dimension exists, so we set it to 1
            M = np.reshape(MPS[i], (MPS[i].shape[0]*MPS[i].shape[1], 1))
            U, S_vector, V = np.linalg.svd(M, full_matrices=False)
            S = np.diag(S_vector)
            MPS[i] = np.reshape(U, (MPS[i].shape[0], MPS[i-1].shape[1]))
        lambda_tensors.append(S)

    lambda_tensors.pop(-1)  # Last element is just the norm
    return MPS, lambda_tensors


### Normalizes the tensor network such that B B_dagger = Identity ###
def right_normalize(MPS):
    lambda_tensors = []
    # Loop from right bound to left bound
    for i in range(len(MPS)-1, -1, -1):
        # Right bound (2 x d)
        if i == len(MPS)-1:
            M = MPS[i].T # Needs to be (d x 2)
            # Dimensions (d x d) (d x 2) (2 x 2)
            U, S_vector, V = np.linalg.svd(M, full_matrices=False)
            S = np.diag(S_vector)
            MPS[i] = V.T # Transpose so that bond dimension is second
            # Update next position M' = MUS
            MPS[i-1] = np.einsum('ijk, jb->ibk', MPS[i-1], U @ S)

        # Inner tensor
        elif i != 0:
            # Collapse right bond and physical dimension (no need to permute)
            M = np.reshape(MPS[i], (MPS[i].shape[0], MPS[i].shape[1]*MPS[i].shape[2]))

            U, S_vector, V = np.linalg.svd(M, full_matrices=False)
            S = np.diag(S_vector)
            
            # Reshape to (left bond, right bond/previous left bond, physical dim)
            MPS[i] = np.reshape(V, (V.shape[0], MPS[i+1].shape[0], MPS[i].shape[2]))

            # Last tensor is a rank-2 tensor
            if i == 1:
                MPS[i-1] = np.einsum('ij, jk->ik', MPS[i-1], U @ S)
            # Other inner tensors are rank-3 tensors
            else:
                MPS[i-1] = np.einsum('ijk, jb->ibk', MPS[i-1], U @ S)

        # Left bound
        elif i == 0:
            M = np.reshape(MPS[i], (1, MPS[i].shape[0]*MPS[i].shape[1]))
            U, S_vector, V = np.linalg.svd(M, full_matrices=False)
            S = np.diag(S_vector)
            MPS[i] = np.reshape(V, (MPS[i].shape[0], MPS[i+1].shape[0]))
        lambda_tensors.append(S)

    lambda_tensors.pop(-1)  # Last element is just the norm
    # lambdas start from right, so need to be reversed
    return MPS, lambda_tensors[::-1]


def contract_outer(bra, ket):
    pos = np.einsum('ij, ib->jb', bra, ket)
    pos = np.reshape(pos, (bra.shape[1]*ket.shape[1]))
    return pos


def contract_inner(bra, ket):
    pos = np.einsum('ijk, abk->iajb', bra, ket)
    pos = np.reshape(pos, (bra.shape[0]*ket.shape[0], bra.shape[1]*ket.shape[1]))
    return pos


### Creates a bond canonical form with left and right normalization on each side of bond ###
def bond_canonical(gamma_tensors, lambda_tensors, bond):
    # Bond i occurs between site i, i+1
    A_tensors = []
    B_tensors = []

    for i in range(0, len(gamma_tensors)):
        if i < bond:
            if i == 0:
                A = gamma_tensors[0]
            else:
                A = np.einsum('ij, jbc->ibc', lambda_tensors[i-1], gamma_tensors[i])
            A_tensors.append(A)
        elif i >= bond:
            if i == len(gamma_tensors)-1:
                B = gamma_tensors[-1]
            else:
                B = np.einsum('ijk, jb->ibk', gamma_tensors[i], lambda_tensors[i])
            B_tensors.append(B)
    MPS = A_tensors + B_tensors
    return MPS


### Creates a site canonical form such as AAAA...M...BBBB ###
### Setting site = N-1 gives us a recreated AA...M state ###
### Setting site = 0 gives us a recreated M...BB state ###
# NOTE: Adding a dummy lambda to left and right could make this more efficient #
def site_canonical(gamma_tensors, lambda_tensors, site):
    # Bond i occurs between site i, i+1
    A_tensors = []
    B_tensors = []

    # Create left canonical matrices
    for i in range(0, len(gamma_tensors)):
        if i < site:
            if i == 0:
                A = gamma_tensors[0]
            else:
                A = np.einsum('ij, jbc->ibc', lambda_tensors[i-1], gamma_tensors[i])
            A_tensors.append(A)

    # Create matrix for optimization
        elif i == site:
            if i == 0:
                M = np.einsum('ij, jb->ib', gamma_tensors[i], lambda_tensors[i])
            elif i == len(gamma_tensors)-1:
                M = np.einsum('ij, aj->ai', lambda_tensors[i-1], gamma_tensors[i])
            else:
                M = np.einsum('ij, jbc, bn->inc', lambda_tensors[i-1], gamma_tensors[i], lambda_tensors[i])

    # Create right canonical matrices
        elif i > site:
            if i == len(gamma_tensors)-1:
                B = gamma_tensors[-1]
            else:
                B = np.einsum('ijk, jb->ibk', gamma_tensors[i], lambda_tensors[i])
            B_tensors.append(B)

    MPS = A_tensors + [M] + B_tensors
    return MPS

In [3]:
### COMPRESSION SWEEPS ###

# TODO: Remove site == len(bra)-1 parts, we never reach last site going left to right
### Left->right sweep ###
def update_site(bra, ket, previous_lambda, site):
    for i in range(0, site):

        # Created L tensor, should end with shape (bondDim_raw, bondDim_compressed)
        if i == 0:  # Left bound
            if site == 1:  # We only have the left bound as the L tensor
                L = np.einsum('ij, ib->bj', bra[i], ket[i])
            else:
                pos = contract_outer(bra[i], ket[i])
                L = pos
        elif i == site-1:  # We need to keep the leg of the bra that connects to the missing tensor
            pos = np.einsum('ijk, abk->iajb', bra[i], ket[i])
            # (ia) will connect to previous L, (b) will conect to M, (j) will be left bond of final tensor
            # -> (Left bound, right bound, final tensor left leg)
            pos = np.reshape(pos, (bra[i].shape[0]*ket[i].shape[0], ket[i].shape[1], bra[i].shape[1]))
            # L has shape (Connects to M, final leg of tensor)
            L = np.einsum('i, ibc->bc', L, pos)

        else:  # Inner sites
            pos = contract_inner(bra[i], ket[i])
            L = np.einsum('i, ij->j', L, pos)

    M = ket[site]

    # Created R tensor, should end with shape (bondDim_raw, bondDim_compressed)
    for i in range(len(bra)-1, site, -1):
        if i == len(bra)-1:
            if i == site+1:
                pos = np.einsum('ij, ib->bj', bra[i], ket[i])
                R = pos
            else:
                pos = contract_outer(bra[i], ket[i])
                R = pos
        elif i == site+1:  # We need to keep the leg of the bra that connects to the missing tensor
            pos = np.einsum('ijk, abk->iajb', bra[i], ket[i])
            # (a) will conect to M, (jb) will connect to previous R, (i) will be right bond of final tensor
            # -> (Left bound, right bound, final tensor left leg)
            pos = np.reshape(pos, (ket[i].shape[0], bra[i].shape[1]*ket[i].shape[1], bra[i].shape[0]))
            R = np.einsum('ijk, j->ik', pos, R)

        else:  # Inner sites
            pos = contract_inner(bra[i], ket[i])
            R = np.einsum('ij, j->i', pos, R)

    if site == 0:
        updated_M = np.einsum('ij, jm->im', M, R)
    elif site == len(bra)-1:
        updated_M = np.einsum('ij, ai->aj', L, M)
    else:
        updated_M = np.einsum('ij, ibc, bm->jmc', L, M, R)

    if updated_M.ndim == 3:
        reshaped_M = np.transpose(updated_M, (0, 2, 1))  # Move left bond and physical dimension together
        reshaped_M = np.reshape(reshaped_M, (reshaped_M.shape[0]*reshaped_M.shape[1], reshaped_M.shape[2]))
        U, S_vector, V = np.linalg.svd(reshaped_M, full_matrices=False)

        A_temp = np.reshape(U, (bra[site].shape[0], bra[site].shape[2], U.shape[1]))
        A_temp = np.transpose(A_temp, (0, 2, 1))

        lambda_inverse = np.linalg.inv(previous_lambda)
        updated_gamma = np.einsum('ij, jbc->ibc', lambda_inverse, A_temp)
    else:  # No need to combine legs
        if site == 0:
            U, S_vector, V = np.linalg.svd(updated_M, full_matrices=False)
            A_temp = U
            updated_gamma = A_temp
        elif site == len(bra)-1:
            temp = np.reshape(updated_M, (updated_M.shape[0]*updated_M.shape[1], 1))
            U, S_vector, V = np.linalg.svd(temp, full_matrices=False)
            A_temp = np.reshape(U, (updated_M.shape[0], updated_M.shape[1]))
            print("Last singular value:", S_vector)
            lambda_inverse = np.linalg.inv(previous_lambda)
            updated_gamma = np.einsum('ij, aj->ai', lambda_inverse, A_temp)
    updated_lambda = np.diag(S_vector)
    
    if site == len(bra)-2:  # Contracting with last site
        updated_next_site = np.einsum('ij, aj->ia', V, bra[site+1])
        return updated_gamma, updated_lambda, updated_next_site
    elif site == len(bra)-1:  # Final site, no need to update
        updated_next_site = []
    else:
        updated_next_site = np.einsum('ij, jbc->ibc', V, bra[site+1])

    return updated_gamma, updated_lambda, updated_next_site

In [4]:
### DECOMPOSITIONS AND REBUILDS ###

### Decomposes our MPS into Gamma matrices on sites, Lambda matrices on bonds ###
### Lambda matrices contain the singular values, final lambda matrix is related to norm ###
### Requires a fully left or right normalized state ###
def vidal_notation(tensors, lambda_tensors, normalization):
    # Trim singular values under a threshold, otherwise inverse is hard to calculate
    #threshold = 10e-8
    #for i in range(0, len(lambda_tensors)):
    #    lambda_tensors[i][lambda_tensors[i] < threshold] = 0

    # Calculate inverse matrices
    lambda_inverse = []
    for i in range(0, len(lambda_tensors)):
        try:
            lambda_inverse.append(np.array(np.linalg.inv(lambda_tensors[i])))
        except:
            print("Lambda inverse cannot be calculated.")
            print(lambda_tensors[i])
    
    if normalization == 'left':  # A tensors as input
        A_tensors = tensors[:]
        # Calculate gamma tensors
        gamma_tensors = []
        gamma_tensors.append(A_tensors[0])  # First gamma is just the first A
        for i in range(1, len(A_tensors)-1):
            gamma = np.einsum('ij, jbc->ibc', lambda_inverse[i-1], A_tensors[i])
            gamma_tensors.append(gamma)
        # Final lambda value is just the final singular value/norm
        gamma = np.einsum('ij, aj->ai', lambda_inverse[-2], A_tensors[-1])
        gamma_tensors.append(gamma)
    
    elif normalization == 'right':  # B tensors as input
        B_tensors = tensors[:]
        # Calculate gamma tensors
        gamma_tensors = []
        gamma = np.einsum('ij, jb->ib', B_tensors[0], lambda_inverse[0])
        gamma_tensors.append(gamma)
        for i in range(1, len(B_tensors)-1):
            gamma = np.einsum('ijk, jb->ibk', B_tensors[i], lambda_inverse[i])
            gamma_tensors.append(gamma)
        # Final lambda value is just the final singular value/norm
        gamma_tensors.append(B_tensors[-1])  # Last gamma is just the last B

    return gamma_tensors, lambda_tensors

In [8]:
### QUANTIFYING METHODS ###

### Calculates scalar product/overlap between two states ###
### <A|B> = N_a*N_b or <A|A> = N^2 = 1/S^2 ###
def scalar_product(A, B):
    pos_0 = np.einsum('ij, ib->jb', A[0], B[0])
    pos_0 = np.reshape(pos_0, (A[0].shape[1]*B[0].shape[1]))
    result = pos_0
    for i in range(1, len(A)-1):
        pos_i = np.einsum('ijk, abk->iajb', A[i], B[i])
        pos_i = np.reshape(pos_i, (A[i].shape[0]*B[i].shape[0], A[i].shape[1]*B[i].shape[1]))
        result = np.einsum('i, ij->j', result, pos_i)
    pos_N = np.einsum('ij, ib->jb', A[-1], B[-1])
    pos_N = np.reshape(pos_N, (A[-1].shape[1]*B[-1].shape[1]))
    result = np.einsum('i, i', result, pos_N)
    return result

def scalar_product_trimmed(A, B, site):
    if site != 0:
        pos_0 = np.einsum('ij, ib->jb', A[0], B[0])
        pos_0 = np.reshape(pos_0, (A[0].shape[1]*B[0].shape[1]))
        result = pos_0
    for i in range(1, site):
        pos_i = np.einsum('ijk, abk->iajb', A[i], B[i])
        pos_i = np.reshape(pos_i, (A[i].shape[0]*B[i].shape[0], A[i].shape[1]*B[i].shape[1]))
        if i == 1 and site != 0:
            result = pos_i
        else:
            result = np.einsum('i, ij->j', result, pos_i)
    pos_N = np.einsum('ij, ib->jb', A[-1], B[-1])
    pos_N = np.reshape(pos_N, (A[-1].shape[1]*B[-1].shape[1]))
    
    if site == 0:
        result = pos_N
    else:
        result = np.einsum('i, i', result, pos_N)
    return result


### Calculates the distance between two states ###
def overlap(A, B):
    overlap = np.sqrt(scalar_product(A, A) + scalar_product(B, B) - scalar_product(A, B) - scalar_product(B, A))
    return overlap

In [22]:
### PARAMETERS ###
#N = rand.randint(3, 20)
#D = rand.randint(10, 15)
#d = rand.randint(2, 5)
N = 50  # Number of particles
D = 5 # Uncompressed state bond dimension
d = 2 # Compressed state bond dimension

In [23]:
raw_state = initialize_random_state(numParticles = N, bondDim = D)

In [24]:
initial_compressed_state = initialize_random_state(numParticles = N, bondDim = d)

In [25]:
### SWEEPS ONLY TO LAST MATRIX ###
print("N:", N, "D:", D, "d:", d)
print("Original distace:", overlap(initial_compressed_state, raw_state))
print("Norm Check:", scalar_product(initial_compressed_state, initial_compressed_state), "\n")
B_tensors, lambda_tensors = right_normalize(initial_compressed_state[:])
print("Left-normalized state distace:", overlap(B_tensors, raw_state))
print("Norm Check:", scalar_product(B_tensors, B_tensors), "\n")
gamma_tensors, lambda_tensors = vidal_notation(B_tensors, lambda_tensors, 'right')

distance = overlap(initial_compressed_state, raw_state)
# Puts M into compressed state, updates gamma and lambda tensors
# LEFT TO RIGHT SWEEP #
for site in range(0, len(raw_state)-1):
    compressed_state_site_canonical  = site_canonical(gamma_tensors, lambda_tensors, site = site)
    gamma_tensors[site], lambda_tensors[site], gamma_tensors[site+1] = update_site(compressed_state_site_canonical[:], raw_state, lambda_tensors[site-1], site = site)

    # Rebuilds the gamma and lambda tensors into a left-normalized state
    compressed_state = site_canonical(gamma_tensors, lambda_tensors, site = N-1)  # Setting site == N-1 gives us the rebuilt state

    print("Site", site+1, "Updated:", overlap(compressed_state, raw_state), "\n")
    if overlap(compressed_state, raw_state) < distance:
       closest_state = compressed_state
       distance = overlap(closest_state, compressed_state)
       print("CLOSEST STATE")

N: 50 D: 5 d: 2
Original distace: 3.133072335476424e+25
Norm Check: 3.43429528131552e+26 

Left-normalized state distace: 3.1330723354781508e+25
Norm Check: 1.0000000000000009 

Site 1 Updated: 1.2359231107083853e+38 

Site 2 Updated: 7.438983629035981e+37 

Site 3 Updated: 3.14003787021503e+37 

Site 4 Updated: 1.5411002282549218e+37 

Site 5 Updated: 7.17399786786679e+36 

Site 6 Updated: 3.3739585742634314e+36 

Site 7 Updated: 1.5832469532847815e+36 

Site 8 Updated: 7.432135133733208e+35 

Site 9 Updated: 3.4886010744091746e+35 

Site 10 Updated: 1.6375538382759096e+35 

Site 11 Updated: 7.686664607564081e+34 

Site 12 Updated: 3.608119164679619e+34 

Site 13 Updated: 1.6936498737631822e+34 

Site 14 Updated: 7.949987537700484e+33 

Site 15 Updated: 3.7317216870052305e+33 

Site 16 Updated: 1.751669020117616e+33 

Site 17 Updated: 8.222328998144359e+32 

Site 18 Updated: 3.8595587007881296e+32 

Site 19 Updated: 1.8116756588885264e+32 

Site 20 Updated: 8.503999872593214e+31 

Sit

In [159]:
overlap(compressed_state, raw_state)

3557407.7140710535

In [31]:
### SWEEPS FULL CHAIN ###

# print("Original distace:", overlap(initial_compressed_state, raw_state))
# print("Norm Check:", scalar_product(initial_compressed_state, initial_compressed_state), "\n")
# B_tensors, lambda_tensors = right_normalize(initial_compressed_state[:])
# print("Left-normalized state distace:", overlap(B_tensors, raw_state))
# print("Norm Check:", scalar_product(B_tensors, B_tensors), "\n")
# gamma_tensors, lambda_tensors = vidal_notation(B_tensors, lambda_tensors, 'right')

# distance = overlap(initial_compressed_state, raw_state)
# # Puts M into compressed state, updates gamma and lambda tensors
# # LEFT TO RIGHT SWEEP #
# for site in range(0, len(raw_state)):
#     compressed_state_site_canonical  = site_canonical(gamma_tensors, lambda_tensors, site = site)
#     if site != len(raw_state)-1:
#         gamma_tensors[site], lambda_tensors[site], gamma_tensors[site+1] = update_site(compressed_state_site_canonical[:], raw_state, lambda_tensors[site-1], site = site)
#     else:
#         #_, _, _ = update_site(compressed_state_site_canonical, raw_state, lambda_tensors[site-1], site = site)
#         gamma_tensors[site], _, _ = update_site(compressed_state_site_canonical, raw_state, lambda_tensors[site-1], site = site)
#     # Rebuilds the gamma and lambda tensors into a left-normalized state
#     compressed_state = rebuild_A_tensors(gamma_tensors, lambda_tensors)

#     print("Site", site, "Updated:", overlap(compressed_state, raw_state), "\n")
#     if overlap(compressed_state, raw_state) < distance:
#         closest_state = compressed_state

In [32]:
overlap(closest_state, raw_state)

6150.511352659728

In [33]:
for tensor in initial_compressed_state:
    print(tensor.shape)

(2, 2)
(2, 2, 2)
(2, 2, 2)
(2, 2, 2)
(2, 2)


In [34]:
for tensor in compressed_state:
    print(tensor.shape)

(2, 2)
(2, 2, 2)
(2, 2, 2)
(2, 2, 2)
(2, 2)


In [35]:
for tensor in gamma_tensors:
    print(tensor)

[[-0.73338451 -0.67981407]
 [-0.67981407  0.73338451]]
[[[-0.00013834 -0.00015267]
  [ 0.00015181 -0.00013742]]

 [[-0.00091469 -0.00115137]
  [ 0.00362553 -0.00376272]]]
[[[-1.23400915e-04 -1.29123345e-04]
  [ 9.05646028e-05 -4.10838969e-05]]

 [[-3.03126116e-02 -3.05218942e-02]
  [ 3.43116696e-02 -5.03327439e-02]]]
[[[-1.75776915e-04 -1.77712400e-04]
  [ 8.72780251e-05 -2.52097686e-04]]

 [[-2.87738235e-02 -2.90286842e-02]
  [ 3.43041242e-02  1.05820908e-02]]]
[[ 0.70966269  0.7045416 ]
 [ 0.7045416  -0.70966269]]


In [36]:
for tensor in lambda_tensors:
    print(tensor)

[[4851.14260329    0.        ]
 [   0.           22.01147774]]
[[4311.48572173    0.        ]
 [   0.           14.83067175]]
[[3154.32106056    0.        ]
 [   0.           15.0490736 ]]
[[3131.42983583    0.        ]
 [   0.           34.5476538 ]]


In [37]:
### CHECK THAT BOND CANONICAL PROPERLY CREATED ###
# test_identity = np.einsum('ij, ib->jb', A_tensors[0], A_tensors[0])
# print("Pos", "1", ":\n", test_identity)
# for i in range(1, len(A_tensors)):
#     # We contract left dimension and physical dimension for each site
#     # (d x d x 2)
#     test_identity = np.einsum('ijk, ibk->jb', A_tensors[i], A_tensors[i])
#     print("Pos", i+1, ":\n", test_identity)

# for i in range(0, len(B_tensors)-1):
#     # We contract right dimension and physical dimension for each site
#     test_identity = np.einsum('ijk, ajk->ia', B_tensors[i], B_tensors[i])
#     print("Pos", i+1+len(A_tensors), ":\n", test_identity)
# test_identity = np.einsum('ij, ib->jb', B_tensors[-1], B_tensors[-1])
# print("Pos", N, ":\n", test_identity)

In [38]:
### BOND CANONICAL WORKS FOR ALL BONDS ###
# for i in range(1, N-1):
#     A_tensors, B_tensors, lambda_bond = bond_canonical(gamma_tensors, lambda_tensors, bond=i)

In [39]:
### MANUAL CHECK A DECOMPOSITION ###
# if (A_tensors[0].all() != gamma_tensors[0].all()):
#     print("Error at tensor 0")
# for i in range(1, len(gamma_tensors)):
#     if i != len(gamma_tensors)-1:  # Inner tensors
#         A_recreated = np.einsum('ij, jbc->ibc', lambda_tensors[i-1], gamma_tensors[i])
#     else:
#         A_recreated = np.einsum('ij, aj->ia', lambda_tensors[-2], gamma_tensors[-1])
#     if (A_tensors[i].all() != A_recreated.all()):
#         print("Error at tensor", i)

In [40]:
### MANUAL CHECK B DECOMPOSITION ###
# if (B_tensors[-1].all() != gamma_tensors[-1].all()):
#     print("Error at tensor -1")
# for i in range(len(A_tensors), len(gamma_tensors)):
#     if i != len(gamma_tensors)-1:  # Inner tensors
#         B_recreated = np.einsum('ijk, jb->ibk', gamma_tensors[i], lambda_tensors[i])
#     if (B_tensors[i-len(A_tensors)].all() != B_recreated.all()):
#         print("Error at tensor", i)

In [41]:
# ### MANUAL CHECK NORM #####
# temp0 = np.einsum('ij, ib->jb', MPS[0], MPS[0])
# temp0 = np.reshape(temp0, (MPS[0].shape[1]**2))

# norm = temp0
# for i in range(1, len(MPS)-1):
#     temp = np.einsum('ijk, abk->iajb', MPS[i], MPS[i])
#     temp = np.reshape(temp, (MPS[i].shape[0]**2, MPS[i].shape[1]**2))
#     norm = np.einsum('i, ij->j', norm, temp)
# temp = np.einsum('ij, ib->jb', MPS[-1], MPS[-1])
# temp = np.reshape(temp, (MPS[-1].shape[1]**2))
# norm = np.einsum('i, i', norm, temp)
# print(np.sqrt(norm))

In [42]:
### MANUAL CHECK LEFT CANONICAL FORM ####
# Information in DMRG Schollwöck 4.4 and Delft MPS lecture
# Goal: Contract left dimension and physical dimension to get identity at all sites

MPS = compressed_state[:]
# First site has left dimension 1, so we contract physical dimension
# (2 x d)

print("A Canonical Check \n")
test_identity = np.einsum('ij, ib->jb', MPS[0], MPS[0])
print("Pos", "0", ":\n", test_identity)
for i in range(1, len(MPS)-1):
    # We contract left dimension and physical dimension for each site
    # (d x d x 2)
    test_identity = np.einsum('ijk, ibk->jb', MPS[i], MPS[i])
    print("Pos", i, ":\n", test_identity)
# Last site has right dimension 1, so result is a singular number
# If all other matrices are identity, it is also the norm
# (2 x d)
test_identity = np.einsum('ij, ij', MPS[-1], MPS[-1])
print("Pos", N-1, ":\n", test_identity)


## MANUAL CHECK RIGHT CANONICAL FORM ####
#Information in DMRG Schollwöck 4.4 and Delft MPS lecture
#Goal: Contract right dimension and physical dimension to get identity at all sites
print("\nB Canonical Check \n")
# First site has right dimension 1, so we contract physical dimension
test_identity = np.einsum('ij, ib->jb', MPS[-1], MPS[-1])
print("Pos", N-1, ":\n", test_identity)
for i in range(len(MPS)-2, 0, -1):
    # We contract right dimension and physical dimension for each site
    test_identity = np.einsum('ijk, ajk->ia', MPS[i], MPS[i])
    print("Pos", i, ":\n", test_identity)
# Last site has right dimension 1, so result is a singular number
# If all other matrices are identity, it is also the norm
test_identity = np.einsum('ij, ij', MPS[0], MPS[0])
print("Pos", 0, ":\n", test_identity)

A Canonical Check 

Pos 0 :
 [[1.00000000e+00 1.66533454e-16]
 [1.66533454e-16 1.00000000e+00]]
Pos 1 :
 [[ 1.00000000e+00 -7.37257477e-17]
 [-7.37257477e-17  1.00000000e+00]]
Pos 2 :
 [[ 1.00000000e+00 -5.55111512e-17]
 [-5.55111512e-17  1.00000000e+00]]
Pos 3 :
 [[1.00000000e+00 1.94289029e-16]
 [1.94289029e-16 1.00000000e+00]]
Pos 4 :
 9807046.357126245

B Canonical Check 

Pos 4 :
 [[9.80585282e+06 0.00000000e+00]
 [0.00000000e+00 1.19354038e+03]]
Pos 3 :
 [[1.32978212 0.50046292]
 [0.50046292 0.67021788]]
Pos 2 :
 [[0.77683902 0.82210403]
 [0.82210403 1.22316098]]
Pos 1 :
 [[1.98572415 0.14626698]
 [0.14626698 0.01427585]]
Pos 0 :
 2.000000000000001


In [30]:
######################## EXPECTATION VALUE #############################################
# E_D_R = calculate_expectation(MPS, MPO, MPS, 'down', 'right')
# E_D_L = calculate_expectation(MPS, MPO, MPS, 'down', 'left')
# E_U_R = calculate_expectation(MPS, MPO, MPS, 'up', 'right')
# E_U_L = calculate_expectation(MPS, MPO, MPS, 'up', 'left')

# # Rounding necessary since sometimes the values are slightly off
# # Most likely due to rounding errors in computation
# if (np.round(E_D_R, 5) == np.round(E_D_L, 5)
#                        == np.round(E_U_R, 5)
#                        == np.round(E_U_L, 5)):
#     print("Expectation value is the same in all directions")
# print(E_D_R, E_D_L, E_U_R, E_U_L)

Expectation value is the same in all directions
3.684460150424682 3.6844601504246834 3.684460150424682 3.6844601504246834


In [23]:
# # ####### CONTRACT HAMILTONIAN LEFT AND RIGHT ########
# temp = contract_horizontal(MPO[0], MPO[1], 'right')
# H_right = contract_horizontal(temp, MPO[2], 'right')
# temp = contract_horizontal(MPO[2], MPO[1], 'left')
# H_left = contract_horizontal(temp, MPO[0], 'left')
# if (H_right.all() == H_left.all()):
#     print("Hamiltonian contracts correctly")

In [24]:
# # ######## EXPECTATION VALUE DOWN, RIGHT ##########
# # Left lattice bra->H->ket
# temp = contract_vertical(MPS[0], MPO[0], 'down')
# left = contract_vertical(temp, MPS[0], 'down')

# # Inner lattice bra->H->ket
# temp = contract_vertical(MPS[1], MPO[1], 'down')
# inner = contract_vertical(temp, MPS[1], 'down')

# # Right lattice bra->H->ket
# temp = contract_vertical(MPS[2], MPO[2], 'down')
# right = contract_vertical(temp, MPS[2], 'down')

# temp = contract_horizontal(left, inner, 'right')
# E_down_right = contract_horizontal(temp, right, 'right')

# ###### EXPECTATION VALUE UP, RIGHT ###########
# # Left lattice ket->H->bra
# temp = contract_vertical(MPS[0], MPO[0], 'up')
# left = contract_vertical(temp, MPS[0], 'up')

# # Inner lattice ket->H->bra
# temp = contract_vertical(MPS[1], MPO[1], 'up')
# inner = contract_vertical(temp, MPS[1], 'up')

# # Right lattice ket->H->bra
# temp = contract_vertical(MPS[2], MPO[2], 'up')
# right = contract_vertical(temp, MPS[2], 'up')

# temp = contract_horizontal(left, inner, 'right')
# E_up_right = contract_horizontal(temp, right, 'right')

# ###### EXPECTATION VALUE UP, LEFT ###########
# # Right lattice bra->H->ket
# temp = contract_vertical(MPS[2], MPO[2], 'up')
# right = contract_vertical(temp, MPS[2], 'up')

# # Inner lattice bra->H->ket
# temp = contract_vertical(MPS[1], MPO[1], 'up')
# inner = contract_vertical(temp, MPS[1], 'up')

# # Left lattice bra->H->ket
# temp = contract_vertical(MPS[0], MPO[0], 'up')
# left = contract_vertical(temp, MPS[0], 'up')

# temp = contract_horizontal(right, inner, 'left')
# E_up_left = contract_horizontal(temp, left, 'left')

# ###### EXPECTATION VALUE DOWN, LEFT ###########
# # Right lattice bra->H->ket
# temp = contract_vertical(MPS[2], MPO[2], 'down')
# right = contract_vertical(temp, MPS[2], 'down')

# # Inner lattice bra->H->ket
# temp = contract_vertical(MPS[1], MPO[1], 'down')
# inner = contract_vertical(temp, MPS[1], 'down')

# # Left lattice bra->H->ket
# temp = contract_vertical(MPS[0], MPO[0], 'down')
# left = contract_vertical(temp, MPS[0], 'down')

# temp = contract_horizontal(right, inner, 'left')
# E_down_left = contract_horizontal(temp, left, 'left')

# ######## EXPECTATION VALUE ZIG-ZAG ##########
# # Left lattice bra->H->ket
# temp = contract_vertical(MPS[0], MPO[0], 'down')
# left = contract_vertical(temp, MPS[0], 'down')

# # Inner lattice ket->H->bra
# temp = contract_vertical(MPS[1], MPO[1], 'up')
# inner = contract_vertical(temp, MPS[1], 'up')

# # Right lattice bra->H->ket
# temp = contract_vertical(MPS[2], MPO[2], 'down')
# right = contract_vertical(temp, MPS[2], 'down')

# temp = contract_horizontal(left, inner, 'right')
# E_zig_zag = contract_horizontal(temp, right, 'right')

# if (E_down_right.all() == E_down_left.all() == E_up_right.all() == E_up_left.all() == E_zig_zag.all()):
#      print("Expectation value contracts in all directions correctly")